
# Full heart EP-mechanics
This example shows you how to consume a full heart model and
set it up for a coupled electromechanical simulation.


# Example setup
before computing the fiber orientation, purkinje network we need to load
the required modules, load a heart model and set up the simulator.

## Perform the required imports
Import the required modules and set relevant paths, including that of the working
directory, model, and ls-dyna executable.



In [ ]:
import os

from pint import Quantity

import ansys.heart.core.models as models
from ansys.heart.simulator.settings.material.ep_material import EPMaterial
from ansys.heart.simulator.settings.material.material import NeoHookean
from ansys.heart.simulator.simulator import DynaSettings, EPMechanicsSimulator

# Example setup
Perform the required imports
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Import the required modules and set relevant paths, including that of the working
directory and generated model



In [ ]:
# accept dpf license aggrement
# https://dpf.docs.pyansys.com/version/stable/getting_started/licensing.html#ref-licensing
os.environ["ANSYS_DPF_ACCEPT_LA"] = "Y"

# specify necessary paths.
# Note that we need to cast the paths to strings to facilitate serialization.
case_file = os.path.join("pyansys-heart", "downloads", "Rodero2021", "01", "01.vtk")
workdir = os.path.join(os.path.dirname(case_file), "FullHeart")


path_to_model = os.path.join(workdir, "heart_model.vtu")

## Load the full heart model



In [ ]:
# instantiate a four chamber model
model: models.FullHeart = models.FullHeart(working_directory=workdir)
model.load_model_from_mesh(path_to_model, path_to_model.replace(".vtu", ".partinfo.json"))

## Instantiate the simulator object
instantiate the simulator and settings appropriately.



In [ ]:
# instantaiate dyna settings of choice
lsdyna_path = r"your_dyna_exe"  # tested with DEV-111820
dyna_settings = DynaSettings(
    lsdyna_path=lsdyna_path, dynatype="intelmpi", platform="wsl", num_cpus=6
)


# instantiate simulator object
simulator = EPMechanicsSimulator(
    model=model,
    dyna_settings=dyna_settings,
    simulation_directory=os.path.join(workdir, "ep-mechanics"),
)

# load default simulation settings
simulator.settings.load_defaults()

# compute fiber orientation in the ventricles and atria
simulator.compute_fibers()
simulator.compute_left_atrial_fiber()
simulator.compute_right_atrial_fiber(appendage=[39, 29, 98])

# switch atria to active
simulator.model.left_atrium.fiber = True
simulator.model.left_atrium.active = True

simulator.model.right_atrium.fiber = True
simulator.model.right_atrium.active = True

## Optionally, we can create more anatomical details.
## Sometimes, it's in favor of convergence rate of mechanical solve

# Extract elements around atrial caps and assign as a passive material
ring = simulator.model.create_atrial_stiff_ring(radius=5)
# material is stiff and value is arbitrarily chosen
ring.meca_material = NeoHookean(rho=0.001, c10=0.1, nu=0.499)
# assign default EP material as for atrial
ring.ep_material = EPMaterial.Active()

# Compute universal coordinates:
simulator.compute_uhc()

# Extract elements around atrialvenricular valves and assign as a passive material
simulator.model.create_stiff_ventricle_base(stiff_material=NeoHookean(rho=0.001, c10=0.1, nu=0.499))

# Estimate the stress-free-configuration
simulator.compute_stress_free_configuration()

# Compute the conduction system
simulator.compute_purkinje()
simulator.compute_conduction_system()

## Start main simulation



In [ ]:
simulator.settings.mechanics.analysis.end_time = Quantity(800, "ms")
simulator.settings.mechanics.analysis.dt_d3plot = Quantity(10, "ms")

simulator.model.save_model(os.path.join(workdir, "heart_fib_beam.vtu"))

<div class="alert alert-info"><h4>Note</h4><p>A constant pressure is prescribed to the atria.
   No circulation system is coupled with the atria.</p></div>



In [ ]:
# start main simulation
simulator.dyna_settings.num_cpus = 10
simulator.simulate()

Result in LS-PrePost



.. video:: ../../_static/images/doc_Christobal01_epmeca_fh.mp4
  :width: 600
  :loop:
  :class: center

